<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b07668de58b8c9d6f4576cc110bb8f10fb26268644ae1d161db4cf89b12969b5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-02 16:09:11
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -37.62 K (-0.26%)
Current PnL: -23.40 L (-15.33%)
CY Booked + Current PnL: -9.51 L (-6.23%)
-------------------
Total profit:  1.56 L
Total loss:  -24.95 L
-------------------
Total Booked + Current PnL: 17.47 L (13.91%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.75%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.81 L (62.37%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.56,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,65.0,M-SC,13.04,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.32,-16.55,19.95,0.09,16777.0,-16682.0,84095.0,95.09,38.0,M-SC,2.99,253.0,-0.99,0.59,10.48,OX40N,NTT,DURABLES
51,MASFIN,398.61,1.24,-2.56,25.25,22.05,24107.0,-2505.0,95475.0,-15.99,58.0,H-SC,6.62,164.0,-0.10,0.67,38.78,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.24,5.99,13.40,20.19,24758.0,10443.0,184763.0,-16.96,48.0,X-MC,6.96,68.0,0.42,1.30,22.01,X40,ATH,INSURANCE
43,ITC,452.00,-0.82,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,41.0,X-LC,1.41,5.0,-0.16,1.38,3.41,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,19.78,-52.87,209.13,45.70,174274.0,-93482.0,83333.0,-13.45,71.0,M-SC,23.57,205.0,-0.54,0.59,20.11,XY24,NTT,TRAVEL
13,BSOFT,831.70,5.06,-26.58,105.38,50.79,108817.0,-37391.0,103262.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.73,20.23,XR,ATH,IT
5,ASIANPAINT,3460.25,3.03,3.26,17.12,20.95,44510.0,8219.0,259987.0,2.18,79.0,X-LC,16.56,31.0,0.18,1.83,40.06,X40,ATH,PAINTS
84,WHIRLPOOL,2270.00,1.98,-19.07,124.77,81.92,115925.0,-21887.0,92911.0,-52.58,24.0,M-SC,3.93,236.0,-0.19,0.65,9.99,XR,NTT,DURABLES
0,5PAISA,593.00,1.87,-36.08,76.62,12.90,107017.0,-78824.0,139672.0,105.53,54.0,H-SC,12.31,161.0,-0.74,0.98,16.14,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-7.15,-20.64,87.07,48.45,138540.0,-41393.0,159113.0,-26.32,13.0,X-MC,8.31,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
18,COFFEEDAY,80.00,-2.46,-40.89,128.83,35.27,86475.0,-46426.0,67123.0,-53.72,30.0,L-SC,15.29,270.0,-0.54,0.47,64.29,XR,NTT,HOTELS
26,GILLETTE,11206.78,-2.33,-1.00,33.76,32.42,84857.0,-2541.0,251355.0,-18.53,40.0,X-SC,8.05,85.0,-0.03,1.77,13.95,X40,ATH,FMCG
8,AWL,485.00,-2.31,-21.07,94.47,53.50,225006.0,-63574.0,238177.0,-61.84,29.0,X-MC,2.53,58.0,-0.28,1.67,5.79,XY24,NTT,FMCG
44,JCHAC,2282.00,-2.23,-30.63,44.15,-0.01,39141.0,-39151.0,88654.0,18052.17,12.0,M-SC,0.73,233.0,-1.00,0.62,0.39,OX40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.20,-0.04,106.81,106.72,131679.0,-53.0,123283.0,-50.44,39.0,H-SC,2.06,139.0,-0.00,0.87,23.33,AR,ATH,MISC
37,IEX,219.00,1.25,0.81,47.44,48.63,95624.0,1615.0,201569.0,-32.92,66.0,H-SC,15.08,136.0,0.02,1.42,12.45,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.67,-0.59,36.43,35.63,75897.0,-1237.0,208337.0,-14.53,46.0,H-MC,3.66,119.0,-0.02,1.46,15.98,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.74,-5.66,120.39,107.91,167688.0,-8357.0,139287.0,-23.39,34.0,M-SC,10.98,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.56,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,65.0,M-SC,13.04,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.32,-16.55,19.95,0.09,16777.0,-16682.0,84095.0,95.09,38.0,M-SC,2.99,253.0,-0.99,0.59,10.48,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.63,-23.02,47.41,13.47,98416.0,-62082.0,207585.0,-68.31,32.0,H-SC,1.96,158.0,-0.63,1.46,5.55,XY24,NTT,PAINTS
67,SIS,528.00,0.79,-25.28,62.29,21.26,51678.0,-28068.0,82964.0,1955.96,43.0,H-SC,3.88,166.0,-0.54,0.58,12.09,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.04,2.61,67.89,72.27,118039.0,4420.0,173868.0,-11.98,60.0,M-LC,3.54,99.0,0.04,1.22,8.94,XR,NTT,IT
37,IEX,219.0,1.25,0.81,47.44,48.63,95624.0,1615.0,201569.0,-32.92,66.0,H-SC,15.08,136.0,0.02,1.42,12.45,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.04,2.61,67.89,72.27,118039.0,4420.0,173868.0,-11.98,60.0,M-LC,3.54,99.0,0.04,1.22,8.94,XR,NTT,IT
37,IEX,219.00,1.25,0.81,47.44,48.63,95624.0,1615.0,201569.0,-32.92,66.0,H-SC,15.08,136.0,0.02,1.42,12.45,XR,NTT,MISC
38,INDIAMART,4810.62,-0.20,-0.04,106.81,106.72,131679.0,-53.0,123283.0,-50.44,39.0,H-SC,2.06,139.0,-0.00,0.87,23.33,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.67,-0.59,36.43,35.63,75897.0,-1237.0,208337.0,-14.53,46.0,H-MC,3.66,119.0,-0.02,1.46,15.98,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.74,-5.66,120.39,107.91,167688.0,-8357.0,139287.0,-23.39,34.0,M-SC,10.98,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-7.15,-20.64,87.07,48.45,138540.0,-41393.0,159113.0,-26.32,13.0,X-MC,8.31,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.95,-37.77,117.27,35.20,93898.0,-48600.0,80070.0,4.98,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
8,AWL,485.00,-2.31,-21.07,94.47,53.50,225006.0,-63574.0,238177.0,-61.84,29.0,X-MC,2.53,58.0,-0.28,1.67,5.79,XY24,NTT,FMCG
55,PGHH,17907.65,-0.68,-5.14,41.01,33.76,78124.0,-10320.0,190500.0,-33.05,31.0,X-MC,4.27,57.0,-0.13,1.34,0.00,X40,ATH,FMCG
54,PAGEIND,51605.07,0.53,-8.13,37.23,26.07,56001.0,-13320.0,150420.0,-32.40,31.0,X-MC,10.24,55.0,-0.24,1.06,0.53,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.82,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,41.0,X-LC,1.41,5.0,-0.16,1.38,3.41,X40,NTT,FMCG
3,ACC,3906.0,0.08,-22.17,110.92,64.15,205413.0,-52761.0,185190.0,-54.57,49.0,X-SC,1.52,82.0,-0.26,1.30,3.88,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.47,0.91,23.66,24.79,60133.0,2297.0,254153.0,-3.69,46.0,X-SC,1.90,86.0,0.04,1.78,24.26,X40N,NTT,MISC
36,ICICIPRULI,790.0,-0.80,2.21,28.13,30.97,50817.0,3911.0,180649.0,-21.83,51.0,X-MC,2.16,75.0,0.08,1.27,15.05,X40,ATH,INSURANCE
66,SIEMENS,4671.5,1.53,-9.70,38.99,25.51,65523.0,-18045.0,168050.0,3.25,68.0,H-LC,2.40,49.0,-0.28,1.18,23.84,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.95,-37.77,117.27,35.20,93898.0,-48600.0,80070.0,4.98,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-7.15,-20.64,87.07,48.45,138540.0,-41393.0,159113.0,-26.32,13.0,X-MC,8.31,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
56,QUESS,424.00,-1.77,-29.46,102.50,42.84,47000.0,-19152.0,45854.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,-0.68,-5.14,41.01,33.76,78124.0,-10320.0,190500.0,-33.05,31.0,X-MC,4.27,57.0,-0.13,1.34,0.00,X40,ATH,FMCG
54,PAGEIND,51605.07,0.53,-8.13,37.23,26.07,56001.0,-13320.0,150420.0,-32.40,31.0,X-MC,10.24,55.0,-0.24,1.06,0.53,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.77,-29.46,102.50,42.84,47000.0,-19152.0,45854.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.33,-47.39,189.48,52.31,144715.0,-68785.0,76375.0,-43.96,35.0,X-SC,4.91,91.0,-0.48,0.54,2.23,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.95,-37.77,117.27,35.20,93898.0,-48600.0,80070.0,4.98,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-1.71,2.25,21.35,24.08,27714.0,2858.0,129808.0,-5.97,43.0,X-SC,5.89,89.0,0.10,0.91,22.31,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.49,-6.92,30.62,21.58,39943.0,-9693.0,130446.0,-53.04,34.0,X-MC,6.00,56.0,-0.24,0.92,15.49,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.07,-14.80,40.00,19.28,117902.0,-51200.0,294756.0,-25.89,58.0,X-LC,6.92,1.0,-0.43,2.07,8.97,X40,ATH,IT
41,INFY,2275.00,-0.19,6.27,45.74,54.87,152082.0,19605.0,332493.0,-17.09,60.0,X-LC,2.48,2.0,0.13,2.34,15.07,X40,BTT,IT
76,TMPV,600.00,-0.56,-14.28,65.86,42.18,156053.0,-39464.0,236946.0,-23.08,35.0,X-LC,2.74,3.0,-0.25,1.66,2.64,XY24,NTT,AUTO
82,VBL,671.64,-0.57,-2.76,39.52,35.67,121379.0,-8709.0,307133.0,-14.36,61.0,X-LC,5.45,4.0,-0.07,2.16,10.72,X40N,ATH,FMCG
43,ITC,452.00,-0.82,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,41.0,X-LC,1.41,5.0,-0.16,1.38,3.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.37,-37.22,111.69,32.89,54133.0,-28740.0,48467.0,-698.49,61.0,L-MC,5.92,259.0,-0.53,0.34,33.47,XR,NTT,BANKS
6,ASIANTILES,137.00,0.45,-14.89,113.46,81.67,90586.0,-13970.0,79840.0,7177.78,45.0,L-SC,19.04,271.0,-0.15,0.56,54.61,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.46,-12.40,59.29,39.54,52543.0,-12539.0,88620.0,-31.43,63.0,M-SC,6.63,220.0,-0.24,0.62,24.96,AR,ATH,AUTO
51,MASFIN,398.61,1.24,-2.56,25.25,22.05,24107.0,-2505.0,95475.0,-15.99,58.0,H-SC,6.62,164.0,-0.10,0.67,38.78,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.03,-18.96,51.81,23.03,75190.0,-33945.0,145126.0,57.89,50.0,H-SC,9.85,167.0,-0.45,1.02,43.99,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.56,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,65.0,M-SC,13.04,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,3.03,3.26,17.12,20.95,44510.0,8219.0,259987.0,2.18,79.0,X-LC,16.56,31.0,0.18,1.83,40.06,X40,ATH,PAINTS
13,BSOFT,831.70,5.06,-26.58,105.38,50.79,108817.0,-37391.0,103262.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.73,20.23,XR,ATH,IT
50,LTIM,7201.88,0.19,12.69,16.84,31.66,46711.0,31229.0,277380.0,8.32,69.0,H-LC,14.47,43.0,0.67,1.95,53.91,X200,ATH,IT
24,EASEMYTRIP,26.40,19.78,-52.87,209.13,45.70,174274.0,-93482.0,83333.0,-13.45,71.0,M-SC,23.57,205.0,-0.54,0.59,20.11,XY24,NTT,TRAVEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.99
1,25,44.81
2,50,75.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.40
MC    29.02
LC    25.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.71
X40      21.27
X40N     11.93
XR        9.67
XY25      9.12
AR        9.10
OX40N     7.55
X200      1.95
SR        0.97
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.53
X-LC    20.28
M-SC    11.95
X-SC     8.17
H-MC     4.77
H-LC     3.13
M-MC     1.38
L-SC     1.38
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.77,-5.49,39.60
IT,13.93,-14.02,72.88
FINANCE,9.38,-16.11,65.18
MISC,7.44,-22.76,73.06
ELECTRICAL,6.02,-9.56,49.69
PAINTS,5.99,-8.30,24.38
INSURANCE,4.40,-0.96,36.54
PHARMA,3.91,-0.47,32.62
AUTO,3.38,-19.33,70.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3092202.0,21
XR,1285740.0,13
AR,1277966.0,10
X40,1014205.0,14
X40N,750123.0,9
OX40N,713154.0,10
XY25,343353.0,6
SR,281432.0,2
MH,75791.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3576878.0,25
M-SC,1356964.0,15
X-MC,1278602.0,16
X-LC,876077.0,11
X-SC,756066.0,8
H-MC,399980.0,3
L-SC,263832.0,3
M-LC,118039.0,1
H-LC,112234.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1236881.0      6
           AR         882974.0      5
M-SC       XY24       772357.0      6
H-SC       XR         770960.0      7
X-MC       X40        451010.0      7
X-LC       X40        384440.0      5
X-MC       XY24       384101.0      3
H-SC       OX40N      328840.0      4
M-SC       OX40N      297543.0      5
X-SC       X40N       297184.0      3
H-SC       SR         281432.0      2
X-SC       XY24       280127.0      3
X-MC       X40N       260069.0      4
X-LC       XY24       226708.0      2
H-MC       AR         207952.0      2
X-LC       X40N       192870.0      2
H-MC       XY24       192028.0      1
X-MC       XY25       183422.0      2
X-SC       X40        178755.0      2
L-SC       XR         177061.0      2
M-SC       XR         165547.0      2
           AR         121517.0      2
M-LC       XR         118039.0      1
L-SC       OX40N       86771.0      1
H-SC       MH          75791.0      1
X-LC       XY25        72059.0      2
H-LC       AR          65523.0      1
L-MC       XR          54133.0      1
M-MC       XY25        48697.0      1
H-LC       X200        46711.0      1
L-LC       XY25        39175.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
